In [21]:
import pandas as pd
import numpy as np
import mysql.connector as conn
import warnings
warnings.filterwarnings('ignore')

In [22]:
mydb=conn.connect(host='localhost', user='root', password='password',auth_plugin='mysql_native_password')

In [23]:
cur=mydb.cursor(buffered=True)

In [24]:
cur.execute('use music_store')

In [25]:
cur.execute('show tables')
table_list=[table[0] for table in cur.fetchall()]

In [26]:
table_list

['album',
 'artist',
 'customer',
 'employee',
 'genre',
 'invoice',
 'invoice_line',
 'media_type',
 'playlist',
 'playlist_track',
 'track']

In [29]:
df=pd.read_sql_query('select * from album',mydb)
df.head()

,album_id,title,artist_id
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [30]:
print(df.shape)
print(df.size)

(347, 3)
1041


In [33]:
cur.execute('select * from album')
cur.description

[('album_id', 3, None, None, None, None, 1, 32768, 63),
 ('title', 252, None, None, None, None, 1, 16, 255),
 ('artist_id', 3, None, None, None, None, 1, 32768, 63)]

In [34]:
column=[i[0] for i in cur.description]
print(column)

['album_id', 'title', 'artist_id']


In [35]:
#Checking for datatypes
for table in table_list:
    print('\nTable:',table)
    column_info=cur.execute(f'''select column_name, data_type from Information_schema.columns where table_schema=
    "music_store" and table_name = "{table}" ''')
    for i in cur.fetchall():
        print(i)


Table: album
('album_id', 'int')
('title', 'text')
('artist_id', 'int')

Table: artist
('artist_id', 'int')
('name', 'text')

Table: customer
('customer_id', 'int')
('first_name', 'text')
('last_name', 'text')
('company', 'text')
('address', 'text')
('city', 'text')
('state', 'text')
('country', 'text')
('postal_code', 'text')
('phone', 'text')
('fax', 'text')
('email', 'text')
('support_rep_id', 'int')

Table: employee
('employee_id', 'int')
('last_name', 'text')
('first_name', 'text')
('title', 'text')
('reports_to', 'int')
('levels', 'text')
('birthdate', 'text')
('hire_date', 'text')
('address', 'text')
('city', 'text')
('state', 'text')
('country', 'text')
('postal_code', 'text')
('phone', 'text')
('fax', 'text')
('email', 'text')

Table: genre
('genre_id', 'int')
('name', 'text')

Table: invoice
('invoice_id', 'int')
('customer_id', 'int')
('invoice_date', 'text')
('billing_address', 'text')
('billing_city', 'text')
('billing_state', 'text')
('billing_country', 'text')
('billing

In [36]:
#Checking for null values
for table in table_list:
    print('\nTable: ',table)
    df_table=pd.read_sql_query(f'select * from {table}',mydb)
    print(df_table.isnull().sum())


Table:  album
album_id     0
title        0
artist_id    0
dtype: int64

Table:  artist
artist_id    0
name         0
dtype: int64

Table:  customer
customer_id       0
first_name        0
last_name         0
company           0
address           0
city              0
state             0
country           0
postal_code       0
phone             0
fax               0
email             0
support_rep_id    0
dtype: int64

Table:  employee
employee_id    0
last_name      0
first_name     0
title          0
reports_to     0
levels         0
birthdate      0
hire_date      0
address        0
city           0
state          0
country        0
postal_code    0
phone          0
fax            0
email          0
dtype: int64

Table:  genre
genre_id    0
name        0
dtype: int64

Table:  invoice
invoice_id             0
customer_id            0
invoice_date           0
billing_address        0
billing_city           0
billing_state          0
billing_country        0
billing_postal_code    0
t